# 00 - Setup

In [11]:
#%%capture
#!pip install datasets transformers seqeval[gpu]
#!pip install wandb -q

# previous code used within google colab, following code for usage in different environment

import sys
!{sys.executable} -m pip install datasets transformers seqeval wandb -q

DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621


In [12]:
import torch
from torch.utils.data import Dataset, DataLoader
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score
from transformers import BertTokenizerFast, BertForTokenClassification, AdamW, get_linear_schedule_with_warmup
import os
import wandb

In [ ]:
!wandb login

wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit: 
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [8]:
from torch import cuda
# check if GPU is available
device = 'cuda' if cuda.is_available() else 'cpu'
print(device)

cpu


In [4]:
id_to_label = {0: 'O',
 1: 'B-EP_POL',
 2: 'I-EP_POL',
 3: 'B-EP_WIRT',
 4: 'I-EP_WIRT',
 5: 'B-EP_FINANZ',
 6: 'I-EP_FINANZ',
 7: 'B-EP_MEDIA',
 8: 'I-EP_MEDIA',
 9: 'B-EP_SCI',
 10: 'I-EP_SCI',
 11: 'B-EP_REL',
 12: 'I-EP_REL',
 13: 'B-EP_KULT',
 14: 'I-EP_KULT',
 15: 'B-EP_MIL',
 16: 'I-EP_MIL',
 17: 'B-EP_NGO',
 18: 'I-EP_NGO',
 19: 'B-EP_MOV',
 20: 'I-EP_MOV',
 21: 'B-EP_OWN',
 22: 'I-EP_OWN',
 23: 'B-EO_POL',
 24: 'I-EO_POL',
 25: 'B-EO_WIRT',
 26: 'I-EO_WIRT',
 27: 'B-EO_FINANZ',
 28: 'I-EO_FINANZ',
 29: 'B-EO_MEDIA',
 30: 'I-EO_MEDIA',
 31: 'B-EO_SCI',
 32: 'I-EO_SCI',
 33: 'B-EO_REL',
 34: 'I-EO_REL',
 35: 'B-EO_KULT',
 36: 'I-EO_KULT',
 37: 'B-EO_MIL',
 38: 'I-EO_MIL',
 39: 'B-EO_NGO',
 40: 'I-EO_NGO',
 41: 'B-EO_MOV',
 42: 'I-EO_MOV',
 43: 'B-P_NAT',
 44: 'I-P_NAT',
 45: 'B-P_ETH',
 46: 'I-P_ETH',
 47: 'B-P_FUNC',
 48: 'I-P_FUNC',
 49: 'B-P_AGE',
 50: 'I-P_AGE',
 51: 'B-P_SOZ',
 52: 'I-P_SOZ',
 53: 'B-P_GEN',
 54: 'I-P_GEN',
 55: 'B-GPE',
 56: 'I-GPE'}

label_to_id = {v: k for k, v in id_to_label.items()}
label_to_id

{'O': 0,
 'B-EP_POL': 1,
 'I-EP_POL': 2,
 'B-EP_WIRT': 3,
 'I-EP_WIRT': 4,
 'B-EP_FINANZ': 5,
 'I-EP_FINANZ': 6,
 'B-EP_MEDIA': 7,
 'I-EP_MEDIA': 8,
 'B-EP_SCI': 9,
 'I-EP_SCI': 10,
 'B-EP_REL': 11,
 'I-EP_REL': 12,
 'B-EP_KULT': 13,
 'I-EP_KULT': 14,
 'B-EP_MIL': 15,
 'I-EP_MIL': 16,
 'B-EP_NGO': 17,
 'I-EP_NGO': 18,
 'B-EP_MOV': 19,
 'I-EP_MOV': 20,
 'B-EP_OWN': 21,
 'I-EP_OWN': 22,
 'B-EO_POL': 23,
 'I-EO_POL': 24,
 'B-EO_WIRT': 25,
 'I-EO_WIRT': 26,
 'B-EO_FINANZ': 27,
 'I-EO_FINANZ': 28,
 'B-EO_MEDIA': 29,
 'I-EO_MEDIA': 30,
 'B-EO_SCI': 31,
 'I-EO_SCI': 32,
 'B-EO_REL': 33,
 'I-EO_REL': 34,
 'B-EO_KULT': 35,
 'I-EO_KULT': 36,
 'B-EO_MIL': 37,
 'I-EO_MIL': 38,
 'B-EO_NGO': 39,
 'I-EO_NGO': 40,
 'B-EO_MOV': 41,
 'I-EO_MOV': 42,
 'B-P_NAT': 43,
 'I-P_NAT': 44,
 'B-P_ETH': 45,
 'I-P_ETH': 46,
 'B-P_FUNC': 47,
 'I-P_FUNC': 48,
 'B-P_AGE': 49,
 'I-P_AGE': 50,
 'B-P_SOZ': 51,
 'I-P_SOZ': 52,
 'B-P_GEN': 53,
 'I-P_GEN': 54,
 'B-GPE': 55,
 'I-GPE': 56}

In [ ]:
import time
import datetime

def format_time(elapsed):
    '''
    Takes a time in seconds and returns a string hh:mm:ss
    '''
    # Round to the nearest second.
    elapsed_rounded = int(round((elapsed)))
    
    # Format as hh:mm:ss
    return str(datetime.timedelta(seconds=elapsed_rounded))

# 00 - Define Paths

In [5]:
gold_data_path = '/data/gold/processed/min_annotations_gold_num.csv'
label_count_path = '/data/gold/processed/count_labels_gold_data.csv'
lex_directory= '/data/lexicon/'

# 01 - Load Data and Entity Count, compute Class Distribution

In [ ]:
data_gold = pd.read_csv(gold_data_path)
label_count = pd.read_csv(label_count_path)

In [ ]:
data_gold.count()

sentence_id    4796
token          4796
label          4796
dtype: int64

In [ ]:
label_count = label_count.groupby(['label']).sum()
label_count = label_count.sort_values(by=['count'])
label_count['l'] = label_count.index
label_count

,count,l
label,,
35,0,35
11,1,11
33,3,33
17,5,17
7,6,7
15,6,15
5,9,5
41,12,41
3,14,3


# 02 - Create Representative Label for each Sample

In [ ]:
import ast
import random

def label_sent(row):
  labels = ast.literal_eval(row)
  l = []
  for label in labels:
    if label != 0:
      if label not in l:
        l.append(label)

  if len(l) == 0:
    return 0
  elif 33 in l:
    return 33
  elif 35 in l:
    return 35
  elif 11 in l:
    return 11
  elif 17 in l:
    return 17
  elif 7 in l:
    return 7
  elif 5 in l:
    return 5
  elif 41 in l:
    return 41
  elif 15 in l:
    return 15
  elif 3 in l:
    return 3
  elif 19 in l:
    return 19
  elif 13 in l:
    return 13
  elif 31 in l:
    return 31
  elif 29 in l:
    return 29
  elif 39 in l:
    return 39
  elif 9 in l:
    return 9
  else:
    random.seed(42)
    return l[random.randint(0, len(l)-1)]

data_gold['repr_label'] = data_gold['label'].apply(label_sent)
data_gold


,sentence_id,token,label,repr_label
0,0,"['Sehr', 'geehrter', 'Herr', 'Präsident', '!',...","[0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 23, 0,...",1
1,1,"['Es', 'gibt', 'nach', 'der', 'derzeitigen', '...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 55,...",55
2,2,"['Obwohl', 'Schengen', 'und', 'Dublin', 'von',...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",9
3,3,"['Erstens', 'wissen', 'wir', 'alle', ',', 'das...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]",0
4,4,"['Der', 'Sachverständige', 'Gerald', 'Knaus', ...","[0, 9, 0, 0, 0, 0, 0, 0, 0, 0, 21, 0, 0, 0]",9
...,...,...,...,...
4791,4791,"['Sie', 'arbeiten', 'hier', ',', 'sie', 'zahle...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]",0
4792,4792,"['Sie', 'tragen', 'dazu', 'bei', ',', 'dass', ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]",0
4793,4793,"['Und', 'Sie', 'von', 'der', 'AfD', 'wollen', ...","[0, 23, 0, 0, 0, 0, 0, 47, 0, 47, 0, 0, 0, 0, ...",23
4794,4794,"['Für', 'uns', 'gilt', ':', 'Wer', 'in', 'unse...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0


In [ ]:
count_repr = data_gold['repr_label'].value_counts()
count_repr

0     1335
23     729
21     653
1      624
47     387
55     303
49     156
51     106
53      98
43      78
45      54
37      36
9       32
39      32
27      30
25      27
29      26
31      18
3       12
13      12
41      11
5        9
19       9
15       6
7        6
17       4
33       2
11       1
Name: repr_label, dtype: int64

# 03 - Resample (only remove Samples with no Entities)


In [ ]:
data_gold = data_gold.drop(data_gold[data_gold.repr_label == 0].index)
data_gold

,sentence_id,token,label,repr_label
0,0,"['Sehr', 'geehrter', 'Herr', 'Präsident', '!',...","[0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 23, 0,...",1
1,1,"['Es', 'gibt', 'nach', 'der', 'derzeitigen', '...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 55,...",55
2,2,"['Obwohl', 'Schengen', 'und', 'Dublin', 'von',...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",9
4,4,"['Der', 'Sachverständige', 'Gerald', 'Knaus', ...","[0, 9, 0, 0, 0, 0, 0, 0, 0, 0, 21, 0, 0, 0]",9
6,6,"['Nun', 'wird', 'laut', 'Masterplan', 'fest', ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 47, 0, 0, 23, 0...",47
...,...,...,...,...
4788,4788,"['Es', 'wäre', 'sehr', 'schön', ',', 'wenn', '...","[0, 0, 0, 0, 0, 0, 0, 51, 0, 0, 0, 0, 0, 0, 0,...",51
4789,4789,"['Wir', 'profitieren', 'von', 'Arbeitnehmerinn...","[0, 0, 0, 47, 0, 47, 0, 0, 0, 0, 0, 0, 0, 0, 0...",47
4790,4790,"['Diese', 'Menschen', 'arbeiten', 'in', 'der',...","[0, 47, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...",47
4793,4793,"['Und', 'Sie', 'von', 'der', 'AfD', 'wollen', ...","[0, 23, 0, 0, 0, 0, 0, 47, 0, 47, 0, 0, 0, 0, ...",23


In [ ]:
import ast
import random

count_ls = {0: 0,
 1: 0,
 2: 0,
 3: 0,
 4: 0,
 5: 0,
 6: 0,
 7: 0,
 8: 0,
 9: 0,
 10: 0,
 11: 0,
 12: 0,
 13: 0,
 14: 0,
 15: 0,
 16: 0,
 17: 0,
 18: 0,
 19: 0,
 20: 0,
 21: 0,
 22: 0,
 23: 0,
 24: 0,
 25: 0,
 26: 0,
 27: 0,
 28: 0,
 29: 0,
 30: 0,
 31: 0,
 32: 0,
 33: 0,
 34: 0,
 35: 0,
 36: 0,
 37: 0,
 38: 0,
 39: 0,
 40: 0,
 41: 0,
 42: 0,
 43: 0,
 44: 0,
 45: 0,
 46: 0,
 47: 0,
 48: 0,
 49: 0,
 50: 0,
 51: 0,
 52: 0,
 53: 0,
 54: 0,
 55: 0,
 56: 0}

def count_labs(row):
  labels = ast.literal_eval(row)
  for label in labels:
    if label != 0:
      count_ls[label] += 1

data_gold['label'].apply(count_labs)
count_ls

{0: 0,
 1: 1623,
 2: 0,
 3: 14,
 4: 0,
 5: 9,
 6: 0,
 7: 6,
 8: 0,
 9: 48,
 10: 0,
 11: 1,
 12: 0,
 13: 21,
 14: 0,
 15: 6,
 16: 0,
 17: 5,
 18: 0,
 19: 15,
 20: 0,
 21: 1866,
 22: 0,
 23: 2378,
 24: 0,
 25: 123,
 26: 0,
 27: 154,
 28: 0,
 29: 33,
 30: 0,
 31: 21,
 32: 0,
 33: 3,
 34: 0,
 35: 0,
 36: 0,
 37: 160,
 38: 0,
 39: 44,
 40: 0,
 41: 12,
 42: 0,
 43: 232,
 44: 0,
 45: 160,
 46: 0,
 47: 1422,
 48: 0,
 49: 465,
 50: 0,
 51: 262,
 52: 0,
 53: 224,
 54: 0,
 55: 1235,
 56: 0}

# 04 - Train/Dev/Test Split
- 60/20/20

In [ ]:
# first, remove data for testing - Combined Model will be tested with same test data
test_split = data_gold.sample(frac = 0.2, random_state = 42)
print(test_split)
data_gold = pd.concat([data_gold, test_split]).drop_duplicates(keep=False)

# create train/dev splits
train_split, validate_split = np.split(data_gold.sample(frac = 1, random_state = 42), [int(.75*len(data_gold))])

print(len(train_split))
print(len(validate_split))
print(len(test_split))

train_split = train_split.reset_index(drop=True)
validate_split = validate_split.reset_index(drop=True)
test_split = test_split.reset_index(drop=True)
train_split

      sentence_id                                              token  \
3405         3405  ['Getrennt', 'davon', 'ist', 'das', 'Thema', '...   
559           559  ['Kein', 'Tarifvertrag', ',', 'meine', 'Damen'...   
4073         4073     ['Sehr', 'geehrter', 'Herr', 'Präsident', '!']   
2417         2417  ['Ich', 'möchte', 'Ihnen', ',', 'liebe', 'Frau...   
4375         4375  ['Das', 'ist', 'besonders', 'für', 'die', 'Kin...   
...           ...                                                ...   
999           999  ['Ja', ',', 'auch', 'wir', 'als', 'Union', 'tr...   
1565         1565  ['Ich', 'weiß', ',', 'dass', 'Sie', 'zumindest...   
1190         1190  ['Ich', 'kann', 'es', 'nur', 'wiederholen', '–...   
4661         4661  ['Nehmen', 'Sie', 'doch', 'einmal', 'folgendes...   
805           805  ['Wir', 'sind', 'schwer', 'getroffen', '.', 'K...   

                                                  label  repr_label  
3405  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...          

,sentence_id,token,label,repr_label
0,2501,"['Es', 'geht', 'darum', ',', 'dass', 'wir', 'd...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 51,...",23
1,2857,"['Es', 'war', 'die', 'AfD-Fraktion', ',', 'die...","[0, 0, 0, 23, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...",23
2,723,"['Sehr', 'geehrte', 'Frau', 'Präsidentin', '!'...","[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",1
3,1608,"['Die', 'Antwort', ',', 'die', 'das', 'Grundge...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",23
4,1863,"['Die', 'Commerzbank', 'ist', 'bis', 'heute', ...","[0, 27, 0, 0, 0, 0, 0, 0, 0, 0, 0, 27, 0, 0, 0...",27
...,...,...,...,...
2071,2894,"['Ehrlich', 'gesagt', ',', 'sehr', 'geehrte', ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",21
2072,1857,"['Der', 'Finanzminister', 'tut', 'so', ',', 'a...","[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",39
2073,3230,"['Dabei', 'möchte', 'ich', 'aber', 'auch', 'ni...","[0, 0, 21, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...",21
2074,899,"['Frau', 'Kollegin', ',', 'kommen', 'Sie', 'bi...","[1, 0, 0, 0, 0, 0, 0, 0, 0]",1


# 05 - Define Parameters, Load Tokenizer

In [ ]:
MAX_LEN = 340
TRAIN_BATCH_SIZE = 16
VALID_BATCH_SIZE = 16
TEST_BATCH_SIZE = 16
EPOCHS = 4
LEARNING_RATE = 2e-05
MAX_GRAD_NORM = 10

tokenizer = BertTokenizerFast.from_pretrained("dbmdz/bert-base-german-uncased")

Downloading:   0%|          | 0.00/59.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/242k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/433 [00:00<?, ?B/s]

# 06 - Define Dataset Class, Create Train-/Dev-/Test-Set and DataLoaders
- Implementation adapted from [this Blog by Chris McCormick](https://mccormickml.com/2019/07/22/BERT-fine-tuning/) and [this Notebook by Niels Rogge](https://colab.research.google.com/github/NielsRogge/Transformers-Tutorials/blob/master/BERT/Custom_Named_Entity_Recognition_with_BERT_only_first_wordpiece.ipynb#scrollTo=DWgnNJrYW2GP)

In [ ]:
training_set = dataset(train_split, tokenizer, MAX_LEN)

In [ ]:
validation_set = dataset(validate_split, tokenizer, MAX_LEN)

In [ ]:
test_set = dataset(test_split, tokenizer, MAX_LEN)

In [ ]:
train_params = {'batch_size': TRAIN_BATCH_SIZE,
                'shuffle': True,
                'num_workers': 0
                }

val_params = {'batch_size': VALID_BATCH_SIZE,
                'shuffle': True,
                'num_workers': 0
                }                

test_params = {'batch_size': TEST_BATCH_SIZE,
                'shuffle': True,
                'num_workers': 0
                }





training_loader = DataLoader(training_set, **train_params)
validation_loader = DataLoader(validation_set, **val_params)
testing_loader = DataLoader(test_set, **test_params)

# 07 - Load BERT Model

In [ ]:
model = BertForTokenClassification.from_pretrained("dbmdz/bert-base-german-uncased", num_labels = len(label_to_id))
model.to(device)

Downloading:   0%|          | 0.00/422M [00:00<?, ?B/s]

Some weights of the model checkpoint at dbmdz/bert-base-german-uncased were not used when initializing BertForTokenClassification: ['cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForTokenClassification were not initialized from the model checkpoint a

BertForTokenClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(31102, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwis

# 08 - Define Optimizer and Scheduler
- Implementation adapted from [this Blog by Chris McCormick](https://mccormickml.com/2019/07/22/BERT-fine-tuning/)

In [ ]:
optimizer = AdamW(model.parameters(),
                  lr = LEARNING_RATE,
                  eps = 1e-8
)

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


In [ ]:
total_steps = len(training_loader) * EPOCHS


scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps = 0,
                                            num_training_steps = total_steps)

# 09 - Define Training/Validation Loop
- Implementation based on [this Blog by Chris McCormick](https://mccormickml.com/2019/07/22/BERT-fine-tuning/) and [this Notebook by Niels Rogge](https://colab.research.google.com/github/NielsRogge/Transformers-Tutorials/blob/master/BERT/Custom_Named_Entity_Recognition_with_BERT_only_first_wordpiece.ipynb#scrollTo=DWgnNJrYW2GP)

In [ ]:
import random

def train(epochs):

    seed_val = 42
    random.seed(seed_val)
    np.random.seed(seed_val)
    torch.manual_seed(seed_val)
    torch.cuda.manual_seed_all(seed_val)

    training_stats = []

    # Measure the total training time for the whole run.
    total_t0 = time.time()

    # For each epoch...
    for epoch_i in range(0, epochs):
    
      # ========================================
      #               Training
      # ========================================
    
      # Perform one full pass over the training set.

      print("")
      print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, EPOCHS))
      print('Training...')

      # Measure how long the training epoch takes.
      t0 = time.time()

      tr_epoch_loss = 0
      nb_tr_examples, nb_tr_steps = 0, 0
      tr_preds, tr_labels = [], []
      # put model in training mode

      model.train()
    
      for idx, batch in enumerate(training_loader):
        
        ids = batch['input_ids'].to(device, dtype = torch.long)
        mask = batch['attention_mask'].to(device, dtype = torch.long)
        labels = batch['labels'].to(device, dtype = torch.long)

        model.zero_grad()  

        
        outputs = model(input_ids=ids, attention_mask=mask, labels=labels)
        loss = outputs[0]
        tr_logits = outputs[1]  

    
        tr_epoch_loss += loss.item()

        nb_tr_steps += 1
        nb_tr_examples += labels.size(0)

        loss_step = tr_epoch_loss/nb_tr_steps
        wandb.log({"Training Loss / Step": loss_step,
                   "Learning Rate / Step": scheduler.get_last_lr()[0]})
        
        if idx % 100==0 and idx != 0:
            print(f"Training loss per 100 training batches: {loss_step}")
            elapsed = format_time(time.time() - t0)
            print(' Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(idx, len(training_loader), elapsed))
           
        # compute training accuracy
        flattened_targets = labels.view(-1) # shape (batch_size * seq_len,)
        active_logits = tr_logits.view(-1, model.num_labels) # shape (batch_size * seq_len, num_labels)
        flattened_predictions = torch.argmax(active_logits, axis=1) # shape (batch_size * seq_len,)
        
        # only compute accuracy at active labels
        active_accuracy = labels.view(-1) != -100 # shape (batch_size, seq_len)
        #active_labels = torch.where(active_accuracy, labels.view(-1), torch.tensor(-100).type_as(labels))
        
        labels = torch.masked_select(flattened_targets, active_accuracy)
        predictions = torch.masked_select(flattened_predictions, active_accuracy)
        
        tr_labels.extend(labels)
        tr_preds.extend(predictions)

        #tmp_tr_accuracy = accuracy_score(labels.cpu().numpy(), predictions.cpu().numpy())
        #tr_accuracy += tmp_tr_accuracy
    
        # gradient clipping
        torch.nn.utils.clip_grad_norm_(
            parameters=model.parameters(), max_norm=MAX_GRAD_NORM
        )
        
        # backward pass
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        scheduler.step()

      tr_epoch_loss = tr_epoch_loss / nb_tr_steps

      print(f"Training loss epoch: {tr_epoch_loss}")
      # Measure how long this epoch took.
      training_time = format_time(time.time() - t0)
      print("  Training epoch took: {:}".format(training_time))
      # ========================================
      #               Validation
      # ========================================
      # After the completion of each training epoch, measure our performance on
      # our validation set.

      print("")
      print("Running Validation...")

      t0 = time.time()

      # Put the model in evaluation mode--the dropout layers behave differently
      # during evaluation.
      model.eval()

      # Tracking variables 
      va_loss, va_accuracy = 0, 0
      nb_va_examples, nb_va_steps = 0, 0
      va_preds, va_labels = [], []



      for idx, batch in enumerate(validation_loader):
        
        ids = batch['input_ids'].to(device, dtype = torch.long)
        mask = batch['attention_mask'].to(device, dtype = torch.long)
        labels = batch['labels'].to(device, dtype = torch.long)

        model.zero_grad()

        with torch.no_grad():
          outputs = model(input_ids=ids, attention_mask=mask, labels=labels)
        loss = outputs[0]
        va_logits = outputs[1]
        va_loss += loss.item()

        nb_va_steps += 1
        nb_va_examples += labels.size(0)
        loss_step = va_loss/nb_va_steps
        wandb.log({"Validation Loss / Step": loss_step})

        if idx % 100==0 and idx != 0:
          #loss_step = va_loss/nb_va_steps
          print(f"Validation loss per 100 validation batches: {loss_step}")
          elapsed = format_time(time.time() - t0)
          print(' Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(idx, len(validation_loader), elapsed))
           
        # compute training accuracy
        flattened_targets = labels.view(-1) # shape (batch_size * seq_len,)
        active_logits = va_logits.view(-1, model.num_labels) # shape (batch_size * seq_len, num_labels)
        flattened_predictions = torch.argmax(active_logits, axis=1) # shape (batch_size * seq_len,)
        
        # only compute accuracy at active labels
        active_accuracy = labels.view(-1) != -100 # shape (batch_size, seq_len)
        #active_labels = torch.where(active_accuracy, labels.view(-1), torch.tensor(-100).type_as(labels))
        
        labels = torch.masked_select(flattened_targets, active_accuracy)
        predictions = torch.masked_select(flattened_predictions, active_accuracy)
        
        va_labels.extend(labels)
        va_preds.extend(predictions)

        tmp_va_accuracy = accuracy_score(labels.cpu().numpy(), predictions.cpu().numpy())
        va_accuracy += tmp_va_accuracy

      va_loss = va_loss / nb_va_steps
      va_accuracy = va_accuracy / nb_va_steps
      print(f"Validation loss epoch: {va_loss}")
      print(f"Validation accuracy epoch: {va_accuracy}")
      # Measure how long this epoch took.
      validation_time = format_time(time.time() - t0)
      print("Validation epoch took: {:}".format(validation_time))

      # Record all statistics from this epoch.
      training_stats.append(
        {
            'epoch': epoch_i + 1,
            'Training Loss': tr_epoch_loss,
            'Valid. Loss': va_loss,
            'Valid. Acc.': va_accuracy,
            'Training Time': training_time,
            'Validation Time': validation_time
        })
      

    v_labels = [id_to_label[id.item()] for id in va_labels]
    v_predictions = [id_to_label[id.item()] for id in va_preds]
   
    pd.set_option('precision', 4)
    df_stats = pd.DataFrame(data=training_stats)
    df_stats = df_stats.set_index('epoch')

    print("")
    print("Training complete!")
    print("Total training took {:} (h:mm:ss)".format(format_time(time.time()-total_t0)))

    return df_stats, v_labels, v_predictions

# 10 - Train Model

In [ ]:
wandb.finish()
wandb.init(settings=wandb.Settings(start_method="thread"))
wandb.watch(model, log="all")
df_stats, v_labels, v_predictions = train(EPOCHS)


wandb: Currently logged in as: donatomonti (use `wandb login --relogin` to force relogin)



======== Epoch 1 / 4 ========
Training...
Training loss per 100 training batches: 0.5659543230096893
 Batch   100  of    130.    Elapsed: 0:01:31.
Training loss epoch: 0.4939258699233715
  Training epoch took: 0:01:58

Running Validation...
Validation loss epoch: 0.18646349249915642
Validation accuracy epoch: 0.9544900654766586
Validation epoch took: 0:00:17

======== Epoch 2 / 4 ========
Training...
Training loss per 100 training batches: 0.13644689004315008
 Batch   100  of    130.    Elapsed: 0:01:33.
Training loss epoch: 0.12749793093938094
  Training epoch took: 0:02:00

Running Validation...
Validation loss epoch: 0.101110294545916
Validation accuracy epoch: 0.9723736080776127
Validation epoch took: 0:00:16

======== Epoch 3 / 4 ========
Training...
Training loss per 100 training batches: 0.08521642362569819
 Batch   100  of    130.    Elapsed: 0:01:33.
Training loss epoch: 0.08366384781323946
  Training epoch took: 0:02:01

Running Validation...
Validation loss epoch: 0.0832404

In [ ]:
df_stats

,Training Loss,Valid. Loss,Valid. Acc.,Training Time,Validation Time
epoch,,,,,
1,0.4939,0.1865,0.9545,0:01:58,0:00:17
2,0.1275,0.1011,0.9724,0:02:00,0:00:16
3,0.0837,0.0832,0.9780,0:02:01,0:00:16
4,0.0715,0.0807,0.9784,0:02:01,0:00:16


# 11 - Evaluate Performance on Dev Set

In [ ]:
l = [v_labels]
p = [v_predictions]

In [ ]:
from seqeval.metrics import classification_report
from seqeval.scheme import IOB2

print(classification_report(l, p, mode='strict', scheme=IOB2))

              precision    recall  f1-score   support

   EO_FINANZ       0.70      0.54      0.61        26
    EO_MEDIA       0.00      0.00      0.00         6
      EO_MIL       0.90      0.60      0.72        30
      EO_MOV       0.00      0.00      0.00         1
      EO_NGO       0.00      0.00      0.00         8
      EO_POL       0.78      0.87      0.82       473
      EO_SCI       0.00      0.00      0.00         5
     EO_WIRT       0.00      0.00      0.00        23
   EP_FINANZ       0.00      0.00      0.00         3
     EP_KULT       0.00      0.00      0.00         2
      EP_MIL       0.00      0.00      0.00         4
      EP_MOV       0.00      0.00      0.00         2
      EP_NGO       0.00      0.00      0.00         1
      EP_OWN       0.99      1.00      0.99       392
      EP_POL       0.74      0.80      0.77       333
      EP_REL       0.00      0.00      0.00         1
      EP_SCI       0.00      0.00      0.00        13
     EP_WIRT       0.00    

/usr/local/lib/python3.7/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


# 12 - Define Test Function
- Implementation adapted from [this Blog by Chris McCormick](https://mccormickml.com/2019/07/22/BERT-fine-tuning/) and [this Notebook by Niels Rogge](https://colab.research.google.com/github/NielsRogge/Transformers-Tutorials/blob/master/BERT/Custom_Named_Entity_Recognition_with_BERT_only_first_wordpiece.ipynb#scrollTo=DWgnNJrYW2GP)

In [ ]:
def test(model, testing_loader):
    # put model in evaluation mode
    model.eval()
    
    test_loss, test_accuracy = 0, 0
    nb_test_examples, nb_test_steps = 0, 0
    test_preds, test_labels = [], []

    t0 = time.time()
    
    with torch.no_grad():
        for idx, batch in enumerate(testing_loader):
            
            ids = batch['input_ids'].to(device, dtype = torch.long)
            mask = batch['attention_mask'].to(device, dtype = torch.long)
            labels = batch['labels'].to(device, dtype = torch.long)
            
            outputs = model(input_ids=ids, attention_mask=mask, labels=labels)
            loss = outputs[0]
            test_logits = outputs[1]
            test_loss += loss.item()

            nb_test_steps += 1
            nb_test_examples += labels.size(0)
        
            if idx % 100==0 and idx != 0:
                loss_step = test_loss/nb_test_steps
                print(f"Test loss per 100 evaluation steps: {loss_step}")
                elapsed = format_time(time.time() - t0)
                print(' Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(idx, len(testing_loader), elapsed))
              
            # compute evaluation accuracy
            flattened_targets = labels.view(-1) # shape (batch_size * seq_len,)
            active_logits = test_logits.view(-1, model.num_labels) # shape (batch_size * seq_len, num_labels)
            flattened_predictions = torch.argmax(active_logits, axis=1) # shape (batch_size * seq_len,)
            
            # only compute accuracy at active labels
            active_accuracy = labels.view(-1) != -100 # shape (batch_size, seq_len)
        
            labels = torch.masked_select(flattened_targets, active_accuracy)
            predictions = torch.masked_select(flattened_predictions, active_accuracy)
            
            test_labels.extend(labels)
            test_preds.extend(predictions)
            
            tmp_test_accuracy = accuracy_score(labels.cpu().numpy(), predictions.cpu().numpy())
            test_accuracy += tmp_test_accuracy

    labels = [id_to_label[id.item()] for id in test_labels]
    predictions = [id_to_label[id.item()] for id in test_preds]
    
    test_loss = test_loss / nb_test_steps
    test_accuracy = test_accuracy / nb_test_steps
    print(f"Test Loss: {test_loss}")
    print(f"Test Accuracy: {test_accuracy}")

    return labels, predictions

# 13 - Evaluate Model on Test Set

In [ ]:
labels, predictions = test(model, testing_loader)

Test Loss: 0.08548994607884776
Test Accuracy: 0.9776344665581143


In [ ]:
l = [labels]
p = [predictions]

In [ ]:
from seqeval.metrics import classification_report
from seqeval.scheme import IOB2

print(classification_report(l, p, mode='strict', scheme=IOB2))

              precision    recall  f1-score   support

   EO_FINANZ       0.76      0.43      0.55        30
    EO_MEDIA       0.00      0.00      0.00         5
      EO_MIL       0.77      0.47      0.59        36
      EO_MOV       0.00      0.00      0.00         4
      EO_NGO       0.00      0.00      0.00        10
      EO_POL       0.76      0.86      0.81       499
      EO_REL       0.00      0.00      0.00         2
      EO_SCI       0.00      0.00      0.00         1
     EO_WIRT       0.00      0.00      0.00        18
   EP_FINANZ       0.00      0.00      0.00         2
     EP_KULT       0.00      0.00      0.00        10
      EP_MIL       0.00      0.00      0.00         1
      EP_MOV       0.00      0.00      0.00         5
      EP_NGO       0.00      0.00      0.00         1
      EP_OWN       0.99      1.00      0.99       350
      EP_POL       0.73      0.76      0.75       329
      EP_SCI       0.00      0.00      0.00         9
     EP_WIRT       0.00    

/usr/local/lib/python3.7/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


# 14 - Save Fine-Tuned Model

In [ ]:
import os

directory = '/trained-bert-baseline'

if not os.path.exists(directory):
    os.makedirs(directory)

# save vocabulary of the tokenizer
tokenizer.save_vocabulary(directory)
# save the model weights and its configuration file
model.save_pretrained(directory)
print('All files saved')


All files saved


In [10]:
model = BertForTokenClassification.from_pretrained('trained-bert-baseline', num_labels = len(label_to_id))
model.to(device)
tokenizer = BertTokenizerFast.from_pretrained('trained-bert-baseline')